# Introduction

Source of files, analysis schema, options

# Paths to input and output files and 3rd party software

In [1]:
### Input files
input_vcf = "data/test_variants.vcf"
promoter_regions = "data/brain_promoters_active.interval_list"
enhancer_regions = "data/brain_enhancers_active.interval_list"

### Output folder
output = "output/"

### Software paths
gatk = "/home/researcher/Programs/gatk-4.1.9.0/gatk"

In [3]:
!mkdir $output

# Select biallelic SNPs located in promoters and enhancers

In [20]:
count_before = !$gatk CountVariants -V $input_vcf
print("Number of variants in the input file:", count_before[-4])

Number of variants in the input file: 40


In [21]:
select_promoter_snps = !$gatk SelectVariants -V $input_vcf -L $promoter_regions --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O "$output"promoter_SNPs.vcf
count_promoter_snps = !$gatk CountVariants -V "$output"promoter_SNPs.vcf
print("Number of biallelic SNPs in promoter regions:", count_promoter_snps[-4])

Number of biallelic SNPs in promoter regions: 11


In [22]:
select_enhancer_snps = !$gatk SelectVariants -V $input_vcf -L $enhancer_regions --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O "$output"enhancer_SNPs.vcf
count_enhancer_snps = !$gatk CountVariants -V "$output"enhancer_SNPs.vcf
print("Number of biallelic SNPs in enhancer regions:", count_enhancer_snps[-4])

Number of biallelic SNPs in enhancer regions: 13


# Annotate with allele frequencies from gnomAD genome